# Fine-Tune a Generative AI Model for Dialogue Summarization

This notebook utilizes the [FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5) model, known for its high-quality instruction-tuned capabilities and ability to summarize text without additional modifications. To enhance the model's performance, a full fine-tuning approach is implemented, and the results are evaluated using ROUGE metrics. Subsequently, the notebook investigates Parameter-Efficient Fine-Tuning (PEFT) techniques to fine-tune the model with a reduced number of parameters. The performance of the PEFT model is then evaluated and compared to the fully fine-tuned model, demonstrating that PEFT's advantages in terms of efficiency often outweigh the potential for slightly lower performance metrics. 

## 1 - Set up Kernel, Load Required Dependencies, Dataset and LLM

### 1.1 - Set up Kernel and Required Dependencies

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==2.5.1 \
    torchdata==0.10.1 --quiet

%pip install \
    transformers==4.47.1 \
    datasets==3.2.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

Import the necessary libraries.

In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

c:\Users\rg255041\AppData\Local\anaconda3\envs\PEFT\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1.2 - Load Dataset and LLM

This notebook experiments with the [DialogSum](https://huggingface.co/datasets/knkarthick/dialogsum) Hugging Face dataset. It contains 10,000+ dialogues with the corresponding manually labeled summaries and topics. 

In [2]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

Load the pre-trained [FLAN-T5 model](https://huggingface.co/docs/transformers/model_doc/flan-t5) and its tokenizer directly from HuggingFace. Notice that this notebook will be using the [small version](https://huggingface.co/google/flan-t5-base) of FLAN-T5. Setting `torch_dtype=torch.bfloat16` specifies the memory type to be used by this model.

In [3]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

c:\Users\rg255041\AppData\Local\anaconda3\envs\PEFT\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rg255041\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
